In [1]:
from __future__ import division

from faculty_hiring.misc.util import *
from faculty_hiring.misc.plotting import *  # Definitions for LABEL_SIZE and such
from faculty_hiring.misc.gaussian_kde import gaussian_kde
from faculty_hiring.parse import faculty_parser, institution_parser
from faculty_hiring.misc.productivity import faculty_at_institution
from faculty_hiring.parse import load
from faculty_hiring.parse.nces import parse_phds_awarded
from faculty_hiring.misc.subfield import topic_descriptions, longer_topic_descriptions, num_topics 

from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats.stats import pearsonr, mannwhitneyu, ttest_ind
from scipy.stats import ttest_ind, ks_2samp, chi2_contingency, mannwhitneyu
from scipy.special import gamma as gammaf
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.cross_validation import StratifiedKFold, LeaveOneOut
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
from collections import Counter
from scipy import interp
import statsmodels.api as sm
import numpy as np
import math
import scipy as sp
import pandas as pd
import os
import mpld3
import palettable

sig_level = 0.05

# File locations
faculty_file = '/Users/allisonmorgan/Documents/faculty_hiring/publication_data/current_data/faculty_cs_CURRENT.txt'
inst_file = '/Users/allisonmorgan/Documents/faculty_hiring/publication_data/current_data/inst_cs_CURRENT.txt'
figures_dir = '/Users/allisonmorgan/Code/src/github.com/allisonmorgan/faculty_hiring/notebooks-productivity/figures/'
#nces_table = '/Users/samway/Documents/Work/ClausetLab/projects/faculty_hiring/data/nces_table.txt'

# (Optional, these are for loading publication profiles)
dblp_dir = '/Users/allisonmorgan/Documents/faculty_hiring/publication_data/profiles_DBLP_Nov16/'

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
inst = institution_parser.parse_institution_records(open(inst_file))
asst_faculty = load.load_assistant_profs(open(faculty_file), inst)
ignore = ['All others', 'UNKNOWN', 'CUNY Graduate Center']

load.load_all_publications(asst_faculty, dblp_dir)

In [3]:
valid_titles = ['Assistant Professor', 'Associate Professor', 'Full Professor']

In [4]:
followup_data = '/Users/allisonmorgan/Code/src/github.com/allisonmorgan/faculty_hiring/notebooks-productivity/Finalized_Spreadsheet_Nov21.tsv'

def load_followup_data(faculty, followup_file):
    """ Loads the follow-up spreadsheet data collected by BuffTurkers in Nov2016 """
    df = pd.read_csv(followup_file, sep='\t')
    profiles_updated = 0
    
    for person in faculty:
        # When collection assignements are created, these fields are written out:
        # person.facultyName, person.place, person.current
        num_records = len(df[df['Faculty name'] == person.facultyName])
        if num_records == 0: # No matches, this is fine
            continue  
        elif num_records == 1: # One match, this is great
            row = df[df['Faculty name'] == person.facultyName]
            if row['Institution (2011)'].iloc[0] != person.place:
                continue
            person['followup_title'] = row['Job Title (2016)'].iloc[0]
            person['followup_location'] = row['Institution (2016)'].iloc[0]
            person['followup_department'] = row['Department (2016)'].iloc[0]
            profiles_updated += 1
        else: # Multiple matches, this... is bad. Blow up.
            raise ValueError('Multiple records detected!')
            
    assert (profiles_updated <= len(df))
    print 'Updated %d profiles!' % (profiles_updated)

load_followup_data(asst_faculty, followup_data)

Updated 555 profiles!


## Compare with Publications

In [70]:
trajectories_position = Counter()
trajectories_location = Counter()
trajectories_moved_and_left = Counter()
for person in asst_faculty:
    if 'followup_title' in person and 'current' in person:
        key = "%s -> %s" % (person['current'], person['followup_title'])
        trajectories_position[key] += 1
        
        key = "%s" % ("STAYED" if person['followup_location'] == person['place'] else "MOVED")
        trajectories_location[key] += 1
        
        if person['followup_location'] != person['place']:
            key = "%s" % ("LEFT ACADEMIA" if person['followup_location'] == "NOT IN ACADEMIA" else "STAYED IN ACADEMIA")
            trajectories_moved_and_left[key] += 1
        
print trajectories_position
print trajectories_location
print trajectories_moved_and_left

Counter({'Assistant Professor -> Associate Professor': 366, 'Assistant Professor -> Assistant Professor': 104, 'Assistant Professor -> NOT TT': 68, 'Assistant Professor -> Full Professor': 17})
Counter({'STAYED': 404, 'MOVED': 151})
Counter({'STAYED IN ACADEMIA': 78, 'LEFT ACADEMIA': 73})


In [71]:
# Number of publications for all current assistant faculty of a university. Adjusted for inflation
# Set limit_by_years_post_hire flag to True to consider only the contributions within the first 5 
# years of a researcher's career.
limit_by_years_post_hire = True
ignore = ['All others', 'UNKNOWN', 'CUNY Graduate Center']
pubs_by_inst = {}
pubs_by_year_by_inst = {}
for (name, inst_data) in inst.items():
    if name in ignore:
        continue
    current_faculty = faculty_at_institution(name, asst_faculty)
    n_pubs = []; n_pubs_by_year = [];
    for person in current_faculty:
        start = person.first_asst_job_year
        if limit_by_years_post_hire:
            end = np.min([2012, person.first_asst_job_year + 5])
        else:
            end = 2012
        career_length = end - start
        
        if person.__contains__('dblp_pubs'):
            contribution = 0
            for pub in person.dblp_pubs:
                if 'year' in pub \
                      and pub['year'] >= start \
                      and pub['year'] < end \
                      and pub['pub_type'] in ['inproceedings', 'article']:
                    contribution += inflation_adjust(pub['year'])
            
            n_pubs.append((person.facultyName, contribution))
            n_pubs_by_year.append((person.facultyName, contribution/float(career_length)))
    if len(n_pubs) > 0:
        pubs_by_inst[name] = n_pubs
        pubs_by_year_by_inst[name] = n_pubs_by_year

In [72]:
# Compare the average publication rate of a person in their first five years, 
# to the average publication rate of faculty within their first five years at
# both the institution they came from and went to
publication_trajectories = []
for person in asst_faculty:
    if 'followup_title' in person and person.__contains__('dblp_pubs'):
        initial_institution = person.current_job()[0]
        final_institution = person['followup_location']
        
        # Get average publication rate of the young faculty at either institution
        if initial_institution == final_institution:
            # Don't consider those people who took promotions at the same institution.
            continue
        # This will skip anyone who left academia (i.e. their final_institution == "NOT TT")
        if pubs_by_year_by_inst.has_key(initial_institution) and pubs_by_year_by_inst.has_key(final_institution):
            # TODO: remove people with count == 0. Those people have zero pubs to their name. This shouldn't happen right?
            initial_counts = [count for (_, count) in pubs_by_year_by_inst[initial_institution] if count > 0]
            initial_pub_rate = np.mean([np.log(count) for count in initial_counts])
            initial_pub_std = np.std([np.log(count) for count in initial_counts])
            
            final_counts = [count for (_, count) in pubs_by_year_by_inst[final_institution] if count > 0]
            final_pub_rate = np.mean([np.log(count) for count in final_counts])
            final_pub_std = np.std([np.log(count) for count in final_counts])
            
            # Get this person's publication rate
            start = person.current_job()[1]
            end = np.min([2012, start + 5])
            career_length = end - start
        
            contribution = 0
            for pub in person.dblp_pubs:
                if 'year' in pub \
                      and pub['year'] >= start \
                      and pub['year'] < end \
                      and pub['pub_type'] in ['inproceedings', 'article']:
                    contribution += inflation_adjust(pub['year'])
            personal_pub_rate = (contribution/float(career_length))
            
            publication_trajectories.append((
                    (np.log(personal_pub_rate), inst[initial_institution]['pi'], inst[final_institution]['pi']), 
                    (initial_pub_rate, initial_pub_std), 
                    (final_pub_rate, final_pub_std)
                ))

In [75]:
print("Number of faculty who changed universities (but stayed in academia)", len(publication_trajectories))
personal_pub_rates = [r for (r, _, _ ) in publication_trajectories]
print("Median personal publicatin rate", np.median(np.exp(r)))

('Number of faculty who changed universities (but stayed in academia)', 78)
('Median personal publicatin rate', 277130757.08205402)


In [76]:
# How many times was the final institution a better fit?
final_raw = 0; num_people_raw = 0; 
final_moved_up = 0; num_people_moved_up = 0;
final_moved_down = 0; num_people_moved_down = 0;
#diffs_mu = []; diffs_sigma = [];
# Everything has alreday been log-ed above ^
for ((personal, initial_rank, final_rank), (initial_mu, initial_sigma), (final_mu, final_sigma)) in publication_trajectories:
    likelihood_of_final = sp.stats.norm.pdf(personal, final_mu, final_sigma)
    likelihood_of_initial = sp.stats.norm.pdf(personal, initial_mu, initial_sigma)
    
    #diffs_mu.append(initial_mu - final_mu)
    #diffs_sigma.append(initial_sigma - final_sigma)
    
    # Did the person make a decrease in rank? Or did they move up?
    if (final_rank - initial_rank) > 0:
        if likelihood_of_final > likelihood_of_initial:
            final_moved_up += 1
        num_people_moved_up += 1
    else: 
        if likelihood_of_final > likelihood_of_initial:
            final_moved_down += 1
        num_people_moved_down += 1

In [77]:
"How many people moved up? {0} How many moved down? {1}".format(num_people_moved_up, num_people_moved_down)

'How many people moved up? 29 How many moved down? 49'

In [78]:
(final_moved_up/num_people_moved_up, final_moved_down/num_people_moved_down)

(0.4482758620689655, 0.5918367346938775)

In [79]:
#fig, ax = plt.subplots(1,1, figsize=SINGLE_FIG_SIZE)
#plt.hist(diffs_mu)
#plt.show()

In [80]:
#fig, ax = plt.subplots(1,1, figsize=SINGLE_FIG_SIZE)
#plt.hist(diffs_sigma, bins=100)
#plt.show()